In [1]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import pytorch_lightning as pl

# graph dl
import networkx as nx
# import torch_geometric
from torch_geometric.nn import conv
from torch_geometric import utils

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For Progress Bar
from tqdm import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

In [13]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn.conv import MessagePassing

X = torch.tensor([[0.0, 0.0, 0.0], [2.0, 1.0, 1.0], [2.0, 2.0, 2.0]])
edge_index = torch.tensor([[0, 1, 2, 0], [1, 2, 0, 2]])
graph = Data(x=X, edge_index=edge_index)

## cheb_conv

In [ ]:
class Chebconv(MessagePassing):
    def __init__(self, in_channels, out_channels, k):
        super().__init__()
        self.linear_set = nn.ModuleList([nn.Linear(in_channels, out_channels, bias=False) for i in range(k)])
        self.k = k

    def _get_norm(self, edge_index):
        edge_index, edge_attr = utils.get_laplacian(edge_index)
        lambda_max = 2  # assume lmbda_max == 2t
        edge_attr = 2 * edge_attr / lambda_max
        row, col = edge_index
        mask = row == col
        edge_attr[mask] =- 1
        return edge_attr, edge_index

    def forward(self, X, edge_index):
        norm, edge_index = self._get_norm(edge_index)
        Z_1 = self.linear_set[0](X)
        Z_2 = self.linear_set[1](self.propagate(edge_index, x=X, norm=norm))
        if self.k == 1:
            return Z_1
        elif self.k == 2:
            return  Z_1 + Z_2
        else:
            H = Z_1 + Z_2
            # loop Z and sum
            for i in range(self.k - 2):
                Z_k = 2 * self.propagate(edge_index, x=Z_2, norm=norm) - Z_1
                H += self.linear_set[i](Z_k)
                Z_1 = Z_2
                Z_2 = Z_k
            return H


    def message(self, x_j, norm):  # how to confirm norm is right
        return norm.reshape(-1, 1) * x_j


### tips
1. use multi k=1 cheb_conv